In [1]:
import numpy as np
import tensorflow as tf
import math
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
mnist = read_data_sets("data", one_hot=True, reshape=False, validation_size=5000)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [2]:
import datetime

def timestamp():
    d = datetime.datetime.now()
    return d.strftime("%Y/%m/%d/%X")

In [3]:
# Parameters
training_epochs = 25
learning_rate = 0.001
batch_size = 100
display_step = 1
logs_path_train = '/tmp/tensorflow_logs/example/train' + timestamp()
logs_path_val = '/tmp/tensorflow_logs/example/val' + timestamp()

In [4]:
# Placeholders
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, [None, 10])

# Variables
W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 4], stddev=0.1))
b1 = tf.Variable(tf.ones([4]))
W2 = tf.Variable(tf.truncated_normal([5, 5, 4, 8], stddev=0.1))
b2 = tf.Variable(tf.ones([8]))
W3 = tf.Variable(tf.truncated_normal([4, 4, 8, 12], stddev=0.1))
b3 = tf.Variable(tf.ones([12]))
W4 = tf.Variable(tf.truncated_normal([7 * 7 * 12, 200], stddev=0.1))
b4 = tf.Variable(tf.ones([200]))
W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
b5 = tf.Variable(tf.ones([10]))

In [5]:
# The model
stride = 1  # output is 28x28
y1 = tf.nn.relu(tf.nn.conv2d(x, W1, strides=[1, stride, stride, 1], padding='SAME') + b1)
stride = 2  # output is 14x14
y2 = tf.nn.relu(tf.nn.conv2d(y1, W2, strides=[1, stride, stride, 1], padding='SAME') + b2)
stride = 2  # output is 7x7
y3 = tf.nn.relu(tf.nn.conv2d(y2, W3, strides=[1, stride, stride, 1], padding='SAME') + b3)

# reshape the output from the third convolution for the fully connected layer
y3_conv = tf.reshape(y3, shape=[-1, 7 * 7 * 12])

y4 = tf.nn.relu(tf.matmul(y3_conv, W4) + b4)
y = tf.matmul(y4, W5) + b5

with tf.name_scope('Loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))

with tf.name_scope('Optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
# Initializing the variables
init = tf.global_variables_initializer()

In [7]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cross_entropy)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [8]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer_train = tf.summary.FileWriter(logs_path_train, 
                                                 graph=tf.get_default_graph())
    summary_writer_val = tf.summary.FileWriter(logs_path_val, 
                                               graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_train_loss = 0.
        avg_val_loss = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
          
        # Loop over all batches
        for i in range(total_batch):
            batch_x_train, batch_y_train = mnist.train.next_batch(batch_size)
            batch_x_val, batch_y_val = mnist.validation.next_batch(batch_size)
            
            # Run optimization op (backprop), cost op (to get loss value) and summary nodes
            # and summary nodes
            a, c, summary = sess.run([optimizer, cross_entropy, merged_summary_op], 
                                     feed_dict={x: batch_x_train, 
                                                y_: batch_y_train})            
            
            c_val, summary_val = sess.run([cross_entropy, merged_summary_op], 
                                          feed_dict={x: batch_x_val, 
                                                     y_: batch_y_val}) 

            # Write logs at every iteration
            summary_writer_train.add_summary(summary, epoch * total_batch + i)
            summary_writer_val.add_summary(summary_val, epoch * total_batch + i)

            # Compute average loss
            avg_train_loss += c / total_batch
            avg_val_loss += c_val / total_batch
            
        # Display logs per epoch step
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1),
                  "train loss", "{:.9f}".format(avg_train_loss),
                  "val loss", "{:.9f}".format(avg_val_loss))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy    
    total_test_batch = mnist.test.num_examples // batch_size
    print('total_test_batch', total_test_batch)    
    acc_test_lst = []
    for step in range(mnist.test.images.shape[0] // total_test_batch):        
        acc_test = acc.eval(
            {x: mnist.test.images[step * total_test_batch:(step + 1) * total_test_batch, :], 
             y_: mnist.test.labels[step * total_test_batch:(step + 1) * total_test_batch, :]}) 
        acc_test_lst.append(acc_test)    
    print("Accuracy:", np.mean(acc_test_lst))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/")

Epoch: 0001 train loss 0.519030024 val loss 0.490417672
Epoch: 0002 train loss 0.139758559 val loss 0.128451929
Epoch: 0003 train loss 0.087371599 val loss 0.084609371
Epoch: 0004 train loss 0.061804112 val loss 0.068177679
Epoch: 0005 train loss 0.049285247 val loss 0.062701552
Epoch: 0006 train loss 0.041229983 val loss 0.058394420
Epoch: 0007 train loss 0.033098495 val loss 0.057881123
Epoch: 0008 train loss 0.028011630 val loss 0.055775380
Epoch: 0009 train loss 0.021774917 val loss 0.057643177
Epoch: 0010 train loss 0.020834223 val loss 0.058370087
Epoch: 0011 train loss 0.017774762 val loss 0.054346654
Epoch: 0012 train loss 0.014144538 val loss 0.057289973
Epoch: 0013 train loss 0.013036005 val loss 0.058730355
Epoch: 0014 train loss 0.013827116 val loss 0.062303040
Epoch: 0015 train loss 0.010177226 val loss 0.060663916
Epoch: 0016 train loss 0.009066836 val loss 0.062783316
Epoch: 0017 train loss 0.011169695 val loss 0.066249867
Epoch: 0018 train loss 0.007604376 val loss 0.06